In [ ]:
import pandas as pd

In [ ]:
urls = {
    '2e2mu2011':'http://opendata.cern.ch/record/5200/files/2e2mu_2011.csv',
    '2e2mu2012':'http://opendata.cern.ch/record/5200/files/2e2mu_2012.csv',
    '4e2011':'http://opendata.cern.ch/record/5200/files/4e_2011.csv',
    '4e2012':'http://opendata.cern.ch/record/5200/files/4e_2012.csv',
    '4mu2011':'http://opendata.cern.ch/record/5200/files/4mu_2011.csv',
    '4mu2012':'http://opendata.cern.ch/record/5200/files/4mu_2012.csv',
}

frames = [pd.read_csv(urls[url]) for url in urls]

fields = [
    'Run', 'Event', 'M',
    'pt1', 'eta1', 'phi1', 'PID1',
    'pt2', 'eta2', 'phi2', 'PID2',
    'pt3', 'eta3', 'phi3', 'PID3',
    'pt4', 'eta4', 'phi4', 'PID4',
]

df = pd.concat(frames)[fields]

df = df[(df.M > 120) & (df.M < 130)]

df.head()

In [ ]:
def row_to_event(i, r):
    
    event = {}
    event['run'] = int(r['Run'])
    event['event'] = int(r['Event'])
    
    event['dataset'] = ''
    event['date_time'] = ''
    
    event['jets'] = []
    event['photons'] = []
    event['electrons'] = []
    event['muons'] = []
    event['MET'] = {}

    nmu = 0
    ne = 0
    
    for i in [1,2,3,4]:
        
        lepton = {
            'pt': round(r['pt'+str(i)], 3),
            'eta': round(r['eta'+str(i)], 3),
            'phi': round(r['phi'+str(i)], 3),
            'charge': -1 if r['PID'+str(i)] > 0 else 1
        }
        
        if abs(r['PID'+str(i)]) == 13:
            event['muons'].append(lepton)
            nmu += 1
        else:
            event['electrons'].append(lepton)
            ne += 1
            
    if nmu == 4:
        event['event_type'] = '4mu'
    if ne == 4:
        event['event_type'] = '4e'
    else:
        event['event_type'] = '2e2mu'
        
    assert (nmu+ne) == 4
    
    return event

In [ ]:
events = [row_to_event(index, row) for index, row in df.iterrows()]

print(events)